In [ ]:
import psycopg2

# Replace these values with your own database credentials
db_host = "192.168.1.2"
db_port = 55432
db_name = "lnc"
db_user = "postgres"
db_password = "postgres"

# Establish a connection to the PostgreSQL database
connection = psycopg2.connect(
    host=db_host,
    port=db_port,
    dbname=db_name,
    user=db_user,
    password=db_password
)

In [3]:
cursor = connection.cursor()

In [4]:
cursor.execute("SELECT version();")
db_version = cursor.fetchone()

In [5]:
print("Connected to PostgreSQL database. Version:", db_version)

Connected to PostgreSQL database. Version: ('PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


In [6]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download el

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 4.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.8/782.8 kB 10.4 MB/s eta 0:00:0031m10.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.7/490.7 kB 9.5 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.1 MB/s eta 0:00:0031m10.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:0031m10.6 MB/s eta 0:00:01
⚠ As of spaCy v3.0, shortcuts like 'el' are deprecated. Please use the
full pipeline package name 'el_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


In [7]:
import spacy
nlp = spacy.load("el_core_news_sm")

In [23]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('nlpaueb/bert-base-greek-uncased-v1', device = 'mps')

No sentence-transformers model found with name /Users/spyros/.cache/torch/sentence_transformers/nlpaueb_bert-base-greek-uncased-v1. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/spyros/.cache/torch/sentence_transformers/nlpaueb_bert-base-greek-uncased-v1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be ex

In [21]:
from tqdm import tqdm
import numpy as np

In [24]:
insert_query = "UPDATE item SET paragraph_embeddings_avg = %s, paragraph_embeddings_max = %s where id = %s"

N=100
with connection.cursor() as cursor, connection.cursor() as bulk_cursor:
    cursor.execute(
        f"""
            SELECT id, title, summary, content 
            FROM item 
            WHERE paragraph_embeddings_avg IS NULL
            OR paragraph_embeddings_max IS NULL
            LIMIT 10000
        """)
    total_rows = cursor.rowcount
    rows_processed = 0
    pbar = tqdm(total=total_rows)
    rows = cursor.fetchmany(size=N)
    while rows:
        data = []
        for row in rows:
            id, title, summary, content = row
            paragraph = title + '. ' + (content if content is not None else summary if summary is not None else '')
            sentences = [sent.text for sent in nlp(paragraph).sents]
            if len(sentences) > 0: 
                sentence_embeddings = model.encode(sentences)
                paragraph_embedding_avg = np.mean(sentence_embeddings, axis=0)
                paragraph_embedding_max = np.max(sentence_embeddings, axis=0) if (len(sentences) > 1) else paragraph_embedding_avg
                data.append((paragraph_embedding_avg.tolist(), paragraph_embedding_max.tolist(), id))
            else:
                print(f"Ignoring empty text, id: {id}, paragraph: {paragraph}")
        
        bulk_cursor.executemany(insert_query, data)
        connection.commit()
        rows_processed += len(rows)
        pbar.update(len(rows))
        print(f"updated {len(rows)} rows, total rows processed: {rows_processed}")
        
        rows = cursor.fetchmany(size=N)
    pbar.close()
connection.commit()

  1%|▌                                                           | 100/10000 [00:57<1:35:02,  1.74it/s]

updated 100 rows, total rows processed: 100


  2%|█▏                                                          | 200/10000 [02:09<1:47:34,  1.52it/s]

updated 100 rows, total rows processed: 200


KeyboardInterrupt: 

In [14]:
import torch
print(torch.backends.mps.is_built())

True


In [23]:
cursor.close()
cursor = connection.cursor()

insert_query = "UPDATE item SET paragraph_embeddings_avg = %s, paragraph_embeddings_max = %s where id = %s"

N=10000
cursor.execute(
    f"""
        SELECT id, title, summary, content 
        FROM item 
        WHERE paragraph_embeddings_avg IS NULL
        OR paragraph_embeddings_max IS NULL
    """)

bulk_cursor = connection.cursor()
row = cursor.fetchone()
i = 0
while row is not None:
    id, title, summary, content = row[0], row[1], row[2], row[3]
    paragraph = title + '. ' + content if content != None else summary if summary != None else ''
    doc = nlp(paragraph)
    sentences = [sent.text for sent in doc.sents]
    sentence_embeddings = model.encode(sentences)
    paragraph_embedding_avg = np.mean(sentence_embeddings, axis=0)
    paragraph_embedding_max = np.max(sentence_embeddings, axis=0) if (len(sentences) > 1) else paragraph_embedding_avg
    
    bulk_cursor.execute(insert_query, (paragraph_embedding_avg.tolist(), paragraph_embedding_max.tolist(), id))
    
    if i == N-1:
        break
    if (i % 100) == 0:
        print(i)
        connection.commit()
        bulk_cursor.close()
        bulk_cursor = connection.cursor()
    i = i + 1
    row = cursor.fetchone()
    
bulk_cursor.close()
cursor.close()
connection.commit()

0


DatatypeMismatch: column "paragraph_embeddings_avg" is of type real[] but expression is of type double precision
LINE 1: UPDATE item SET paragraph_embeddings_avg = 'NaN'::float, par...
                                                   ^
HINT:  You will need to rewrite or cast the expression.


In [ ]:
paragraph_embeddings = np.array(paragraph_embedding_list)

In [ ]:
faiss.normalize_L2(paragraph_embeddings)

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(paragraph_embeddings.shape[1]))

In [ ]:
index.add_with_ids(paragraph_embeddings, np.array(ids))

In [ ]:
def nearest_neighbors(paragraph):
    print(paragraph)
    sentences = paragraph.strip().split('.')
    sentence_embeddings = model.encode(sentences)
    paragraph_embedding = np.array([np.mean(sentence_embeddings, axis=0)])
    faiss.normalize_L2(np.array([paragraph_embedding]))
    k = 3
    distances, neighbors = index.search(paragraph_embedding, k)
    print(neighbors[0])
    for n in neighbors[0]: print(paragraph_list[n])
    

In [ ]:
nearest_neighbors(paragraph_list[3])

In [ ]:
index_filepath = '/home/spyros/Temp/test.faiss'
faiss.write_index(index, index_filepath)